In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import numpy as np
from typing import List
import matplotlib.pyplot as plt
from mtt.models import ConvEncoderDecoder

rng = np.random.default_rng()

In [3]:
from mtt.sensor import Sensor
from mtt.simulator import Simulator
from mtt.data import OnlineDataset

init_simulator = lambda: Simulator(
    max_targets=10,
    p_initial=4,
    p_birth=2,
    p_survival=0.95,
    sigma_motion=0.1,
    sigma_initial_state=(3.0, 1.0, 1.0),
    max_distance=1e6,
)
init_sensor = lambda: Sensor(position=(1, 1), noise=(0.2, 0.1), p_detection=0.9)
dataset = OnlineDataset(n_steps=100, length=20, img_size=256, init_simulator=init_simulator, init_sensor=init_sensor)

In [5]:
model = ConvEncoderDecoder()
train_loader = torch.utils.data.DataLoader(dataset, batch_size=4)

logger = TensorBoardLogger(save_dir="../logs", name="tensorboard")
callbacks = [
    ModelCheckpoint(monitor="train/loss", dirpath="../logs/checkpoints"),
]

trainer = pl.Trainer(
    logger=logger,
    callbacks=callbacks,
    auto_lr_find=True,
    precision=32,
    gpus=1,
    max_epochs=10,
    default_root_dir="../logs",
)
trainer.fit(model, train_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 7.1 M 
1 | decoder | Sequential | 7.1 M 
2 | hidden  | Sequential | 1.6 M 
---------------------------------------
15.8 M    Trainable params
0         Non-trainable params
15.8 M    Total params
63.297    Total estimated model params size (MB)


Epoch 0: : 17it [00:12,  1.34it/s, loss=3.03e+08, v_num=17]


RuntimeError: CUDA out of memory. Tried to allocate 640.00 MiB (GPU 0; 7.79 GiB total capacity; 3.88 GiB already allocated; 144.00 MiB free; 4.89 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF